### Capturing word correlation in input data

In [1]:
import numpy as np
onehots = {}
onehots['cat'] = np.array([1,0,0,0])
onehots['the'] = np.array([0,1,0,0])
onehots['dog'] = np.array([0,0,1,0])
onehots['sat'] = np.array([0,0,0,1])

sentence = ['the','cat','sat']
x = onehots[sentence[0]]+onehots[sentence[1]]+onehots[sentence[2]]

print("Sentence Encoding:" + str(x))

Sentence Encoding:[1 1 0 1]
